##### Quiz) 위키피디아에서 제공되는 지진 데이터를 활용한 Q&A 챗봇을 만드시오.

##### 조건)
1. 거대 언어 모델(llm) : gpt-4o-mini
1. 임베딩 모델 : BAAI/bge-m3
1. 문서 : 위키피디아에서 제공하는 지진 내용 전체 (WikipediaLoader 사용, 한국어, doc_content_chars_max 40000)
1. 텍스트 분리(splitter) : CharacterTextSplitter 사용 (chunk_size=1000, chunk_overlap=100)
1. 벡터DB : Chroma
1. 검색기(retriever) : 기본값으로 하되 최대 검색 청크 수 k=3 으로 설정
1. 프롬프트 템플릿 : langchain-ai/retrieval-qa-chat 활용
1. 출력 파서 : StrOutputParser

##### 질문)
1. 지진파의 종류에 대해서 알려줘
1. 지금까지 있었던 지진 중 가장 규모가 큰 것은?
1. 지진을 주제로 한 영화는 어떤 게 있어?

##### 답변) 아래 내용이 포함되면 정답으로 간주
1. P파, S파, 표면파
1. 칠레 발디비아 지진
1. 대지진, 일본 침몰, 2012

In [1]:
# 1. 거대 언어 모델(llm) : gpt-4o-mini
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")

/Users/ryu/dev-study/llm/langchain/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. 임베딩 모델 : BAAI/bge-m3
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [3]:
# 3. 문서 : 위키피디아에서 제공하는 지진 내용 전체 (WikipediaLoader 사용, 한국어, doc_content_chars_max 40000)
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(
  query="지진", 
  lang="ko", 
  load_max_docs=1, 
  doc_content_chars_max=40000,
)

In [4]:
# 4. 텍스트 분리(splitter) : CharacterTextSplitter 사용 (chunk_size=1000, chunk_overlap=100)
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
  chunk_size=1000, # 줄바꿈 기준으로 나누기 위해서 0으로 처리 하려고 했으나 0 이상이어야함
  chunk_overlap=100
)

docs = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 1268, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1451, which is longer than the specified 1000
Created a chunk of size 2325, which is longer than the specified 1000
Created a chunk of size 1052, which is longer than the specified 1000
Created a chunk of size 1213, which is longer than the specified 1000
Created a chunk of size 1260, which is longer than the specified 1000
Created a chunk of size 1317, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000
Created a chunk of size 1998, which is longer than the specified 1000
Created a chunk of size 1141, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000


In [5]:
# 5. 벡터DB : Chroma
from langchain_chroma import Chroma
db = Chroma.from_documents(documents=docs, embedding=hf_embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [6]:
# 6. 검색기(retriever) : 기본값으로 하되 최대 검색 청크 수 k=3 으로 설정
retriever = db.as_retriever(
  search_type="mmr",
  search_kwargs={"k": 3, "fetch_k": 20, "lambda_mult": 0.5}
)

In [7]:
# 7. 프롬프트 템플릿 : langchain-ai/retrieval-qa-chat 활용
from langchain import hub
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
prompt_template = hub.pull("langchain-ai/retrieval-qa-chat")

/Users/ryu/dev-study/llm/langchain/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
# 8. 출력 파서 : StrOutputParser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [9]:
from langchain_core.runnables import RunnablePassthrough

data = {
  "context" : retriever,
  "input" : RunnablePassthrough()
}

# Chain
chain = data | prompt_template | llm | parser

In [ ]:
# 결과 확인
# 질문)
# 1. 지진파의 종류에 대해서 알려줘
# result = chain.invoke("지진파의 종류에 대해서 알려줘")
# 2. 지금까지 있었던 지진 중 가장 규모가 큰 것은?
# result = chain.invoke("지금까지 있었던 지진 중 가장 규모가 큰 것은?")
# 3. 지진을 주제로 한 영화는 어떤 게 있어?
result = chain.invoke("지진을 주제로 한 영화는 어떤 게 있어?")

# 답변)
result
# 1. P파, S파, 표면파 (OK)
# 2. 칠레 발디비아 지진 (OK)
# 3. 대지진, 일본 침몰, 2012 (OK)

'지진을 주제로 한 영화로는 다음과 같은 작품들이 있습니다:\n\n1. 《대지진》(1974년) - 1971년 샌페르난도 지진에서 영감을 받아 만들어진 영화입니다.\n2. 《일본 침몰》(1973년) - 고마쓰 사쿄의 소설을 원작으로 한 영화입니다.\n3. 《2012》(2009년) - 롤란트 에머리히 감독의 영화로, 대규모 지진과 재난을 다루고 있습니다.\n\n이 외에도 지진을 다룬 다양한 영화들이 있습니다.'